### 1. pandas 및 pickle 불러오기

In [518]:
import pandas as pd
import os

num_to_label pickle 에서 불러오려면 아래 셀 uncomment 후 경로 지정

In [519]:
# import pickle
# with open("../code_baseline/dict_num_to_label.pkl",'rb') as fr:
#     num_to_label = pickle.load(fr)
# print(num_to_label)

num_to_label dict 직접 만들기

In [520]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}

### 2. 앙상블 재료들 DF로 불러오기

In [521]:
# 본인이 앙상블할 재료 submission.csv들로 경로 변경
    
if os.path.isdir('./ensemble_result')==False:
    os.mkdir('./ensemble_result')
folder_name = './ensemble_soft'
# folder_name = './ensemble2'
file_list = os.listdir(folder_name)
ens_list =[]
file_name = ""
print(file_list)
for file in file_list:
    # if 'train_14_roberta' in file or 'train_21_seed_159' in file:
    #     continue
    if 'train_21_seed_159' in file:
        continue
    
    print(file)
    file_name+="_"+file[6:12]
    ens1= pd.read_csv(f'{folder_name}/{file}')
    ens_list.append(ens1)

['train_21_seed_159.csv', 'train_14_roberta-large.csv', 'train_seo_02.csv', 'train_13_roberta-large.csv', 'train_19_roberta-large_lr.csv', 'train_sub_69.csv', 'train_seo_01.csv']
train_seo_02.csv
train_13_roberta-large.csv
train_19_roberta-large_lr.csv
train_sub_69.csv
train_seo_01.csv


In [522]:
# 잘 import 되었나 길이 확인
for i in ens_list:
    print(len(i))


7765
7765
7765
7765
7765


두 모델이 다르게 예측한 pred_label의 개수 확인


In [523]:
# result_comparison = ens1['pred_label'] != ens2['pred_label']
# count_true = result_comparison.sum()
# print(f"다른 예측 개수 : {count_true}")

### 3. 확률 계산을 위해 'probs' 칼럼 float 리스트로 만들어서 새 칼럼에 추가

In [524]:
for num, ens in enumerate(ens_list):
    ens['probs_float'] = ens['probs'].apply(lambda x: list(map(float, x[1:-1].split(', '))))
    ens_list[num]=ens


In [525]:
# probs_float 칼럼이 잘 생겼음을 확인
# ens1.head()
ens_list[1].head()

,id,pred_label,probs,probs_float
0,0,org:product,"[0.08556289970874786, 0.015076791867613792, 0....","[0.08556289970874786, 0.015076791867613792, 0...."
1,1,per:alternate_names,"[0.03748128563165665, 0.00022106598771642894, ...","[0.03748128563165665, 0.00022106598771642894, ..."
2,2,no_relation,"[0.9992592930793762, 2.886631955334451e-05, 4....","[0.9992592930793762, 2.886631955334451e-05, 4...."
3,3,no_relation,"[0.9594841599464417, 0.00022557287593372166, 0...","[0.9594841599464417, 0.00022557287593372166, 0..."
4,4,org:top_members/employees,"[0.3767693340778351, 0.616115152835846, 0.0003...","[0.3767693340778351, 0.616115152835846, 0.0003..."


In [526]:
# ens2.head()

### 4. copy를 통해 앙상블 출력용 DataFrame의 뼈대 만들기

In [527]:
ens_result = ens1.copy()
ens_result.head()

,id,pred_label,probs,probs_float
0,0,org:members,"[0.008137032389640808, 0.001768831629306078, 0...","[0.008137032389640808, 0.001768831629306078, 0..."
1,1,per:title,"[0.001582783879712224, 9.086904901778325e-05, ...","[0.001582783879712224, 9.086904901778325e-05, ..."
2,2,no_relation,"[0.9991855025291443, 2.857174513337668e-05, 2....","[0.9991855025291443, 2.857174513337668e-05, 2...."
3,3,no_relation,"[0.9695386290550232, 0.0004900097264908254, 0....","[0.9695386290550232, 0.0004900097264908254, 0...."
4,4,no_relation,"[0.8111861348152161, 0.1833547204732895, 0.000...","[0.8111861348152161, 0.1833547204732895, 0.000..."


### 5. 앙상블 재료들의 확률끼리 numpy 이용해서 평균 산출

In [528]:
import numpy as np

# 실행 시 오류 날텐데 무시 가능..
for i in range(len(ens_result)):
    li = []
    for ens in ens_list:
        li.append(ens['probs_float'][i])
    
    ens_result['probs_float'][i] = np.mean(li, axis=0)

<ipython-input-528-c1eddda2b0bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_result['probs_float'][i] = np.mean(li, axis=0)
<ipython-input-528-c1eddda2b0bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_result['probs_float'][i] = np.mean(li, axis=0)
<ipython-input-528-c1eddda2b0bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ens_result['probs_float'][i] = np.mean(li, axis=0)
<ipython-input-528-c1eddda2b0bb>:9: Sett

기존 probs 칼럼도 평균된 probs_float 값으로 바꿔주기

In [529]:
ens_result['probs'] = ens_result['probs_float'].apply(lambda x: '[' + ', '.join(map(str, x)) + ']')

### 6. 가장 높은 확률을 가진 확률의 index 추출 후, num_to_label 로 pred_label 칼럼 변경

In [530]:
ens_result['pred_label'] = ens_result['probs_float'].apply(lambda x:num_to_label[np.argmax(x)])

In [531]:
for ens in ens_list:
    diff_with_ens1 = ens_result['pred_label'] != ens['pred_label']
    diff_ens1_count = diff_with_ens1.sum()
    print(f"ens 기준 달라진 예측 개수 : {diff_ens1_count}")

ens 기준 달라진 예측 개수 : 443
ens 기준 달라진 예측 개수 : 514
ens 기준 달라진 예측 개수 : 423
ens 기준 달라진 예측 개수 : 519
ens 기준 달라진 예측 개수 : 440


In [532]:
# 형식 맞추기 위해 probs_float 드롭
ens_result = ens_result[['id','pred_label','probs']]

In [533]:
preds = list(ens_result['pred_label'])
labels = list(pd.read_csv('../data/dataset/test/dev_final.csv')['label'])
label_list = ['org:top_members/employees', 'org:members',
    'org:product', 'per:title', 'org:alternate_names',
    'per:employee_of', 'org:place_of_headquarters', 'per:product',
    'org:number_of_employees/members', 'per:children',
    'per:place_of_residence', 'per:alternate_names',
    'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
    'per:spouse', 'org:founded', 'org:political/religious_affiliation',
    'org:member_of', 'per:parents', 'org:dissolved',
    'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
    'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
    'per:religion']

In [534]:
from sklearn.metrics import precision_recall_fscore_support
print(file_name)
precision, recall, f1, none = precision_recall_fscore_support(labels, preds, average='micro', labels=label_list)
print(f"F1: {f1 * 100}\nPrecision: {precision * 100}\nRecall: {recall * 100}")

_seo_02_13_rob_19_rob_sub_69_seo_01
F1: 76.0306807286673
Precision: 76.15236875800257
Recall: 75.90938098276962


In [535]:
ens_result.head()

,id,pred_label,probs
0,0,org:product,"[0.06680897045880556, 0.0055718755000270905, 0..."
1,1,per:alternate_names,"[0.016666355030611157, 0.0002360459984629415, ..."
2,2,no_relation,"[0.9989360928535461, 3.3617886947467925e-05, 4..."
3,3,no_relation,"[0.9582719326019287, 0.00035677631676662714, 0..."
4,4,no_relation,"[0.7489556610584259, 0.24428638175595552, 0.00..."


### 7. 제출 가능한 형태의 csv로 출력

In [536]:
# ens_result.to_csv(f'./save_best/ens_{file_name}.csv', index=False)